# Snake

In [42]:
import environments_fully_observable 
import environments_partially_observable
import numpy as np
from  tqdm import trange
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from agent import *
tf.random.set_seed(0)
random.seed(0)
import math
np.random.seed(0)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Environment definition

In [ ]:
%matplotlib inline
# function to standardize getting an env for the whole notebook
def get_env(n=1000):
    # n is the number of boards that you want to simulate parallely
    # size is the size of each board, also considering the borders
    # mask for the partially observable, is the size of the local neighborhood
    size = 7
    e = environments_fully_observable.OriginalSnakeEnvironment(n, size)
    # or environments_partially_observable.OriginalSnakeEnvironment(n, size, 2)
    return e
env_ = get_env()
GAMMA = .9
ITERATIONS = 5000

fig,axs=plt.subplots(1,min(len(env_.boards), 5), figsize=(10,3))
for ax, board in zip(axs, env_.boards):
    ax.get_yaxis().set_visible(False)
    ax.get_xaxis().set_visible(False)
    ax.imshow(board, origin="lower")

In [ ]:
def bfs_alg(curr_state):
    

## Model

## Training

In [ ]:
reward_history_agent=[0]
EPOCHS=1500
env_ = get_env()
agent=new_agent(env=env_)
for e in range(EPOCHS):
    if e%50==0:
        print(f"{e}/{EPOCHS} - {np.mean(reward_history_agent[-30:]) or 0}", end="\n")
    state=env_.to_state()
    state=tf.reshape(state, (1000, -1))

    original_probs = agent.actor(state)
    pred = original_probs / tf.reduce_sum(original_probs, axis=-1, keepdims=True)
    samples = tf.random.categorical(tf.math.log(pred), 1, dtype=tf.int32)[:, 0]
    actions=tf.reshape(samples, (samples.shape[0],1))

    masks=env_.check_actions(actions)

    rewards= env_.move(actions)
    new_state = tf.constant(env_.to_state())
    new_state=tf.reshape(new_state, (1000, -1))

    agent.learn(state, new_state, samples, rewards) # this optimie the policy function tarting from information sampled
    if e > 50: reward_history_agent.append(np.mean(rewards))
plt.plot(reward_history_agent)

In [43]:
reward_history_agent=[0]
EPOCHS=1500
env_ = get_env()
agent=TRPO_agent(env=env_)
for e in range(EPOCHS):
    if e%50==0:
        print(f"{e}/{EPOCHS} - {np.mean(reward_history_agent[-30:]) or 0}", end="\n")
    state=env_.to_state()
    state=tf.reshape(state, (1000, -1))

    original_probs = agent.actor(state)
    pred = original_probs / tf.reduce_sum(original_probs, axis=-1, keepdims=True)
    samples = tf.random.categorical(tf.math.log(pred), 1, dtype=tf.int32)[:, 0]
    actions=tf.reshape(samples, (samples.shape[0],1))

    masks=env_.check_actions(actions)

    rewards= env_.move(actions)
    new_state = tf.constant(env_.to_state())
    new_state=tf.reshape(new_state, (1000, -1))

    agent.learn(state, new_state, samples, rewards) # this optimie the policy function tarting from information sampled
    if e > 50: reward_history_agent.append(np.mean(rewards))
plt.plot(reward_history_agent)

0/1500 - 0


KeyboardInterrupt: 

In [ ]:
reward_history_conv=[0]
EPOCHS=1500
env_ = get_env()
agent=conv_agent(env=env_)
for e in range(EPOCHS):
    if e%50==0:
        print(f"{e}/{EPOCHS} - {np.mean(reward_history_conv[-30:]) or 0}", end="\n")
    state=env_.to_state()
    #state=tf.reshape(state, (1000, -1))

    original_probs = agent.actor(state)
    pred = original_probs / tf.reduce_sum(original_probs, axis=-1, keepdims=True)
    samples = tf.random.categorical(tf.math.log(pred), 1, dtype=tf.int32)[:, 0]
    actions=tf.reshape(samples, (samples.shape[0],1))

    masks=env_.check_actions(actions)

    rewards= env_.move(actions)
    new_state = tf.constant(env_.to_state())
    #new_state=tf.reshape(new_state, (1000, -1))

    agent.learn(state, new_state, samples, rewards) # this optimie the policy function tarting from information sampled
    if e > 50: reward_history_conv.append(np.mean(rewards))
plt.plot(reward_history_conv)

In [ ]:
reward_history_conv=[0]
EPOCHS=1000
env_ = get_env()
agent=Duelling(env=env_)
for e in range(EPOCHS):
    if e%50==0:
        print(f"{e}/{EPOCHS} - {np.mean(reward_history_conv[-30:]) or 0}", end="\n")
    state=env_.to_state()
    #state=tf.reshape(state, (1000, -1))

    original_probs = agent.model.call(state)
    pred = original_probs / tf.reduce_sum(original_probs, axis=-1, keepdims=True)
    samples = tf.random.categorical(tf.math.log(pred), 1, dtype=tf.int32)[:, 0]
    actions=tf.reshape(samples, (samples.shape[0],1))

    masks=env_.check_actions(actions)

    rewards= env_.move(actions)
    new_state = tf.constant(env_.to_state())
    #new_state=tf.reshape(new_state, (1000, -1))

    agent.learn(state, new_state, samples, rewards) # this optimie the policy function tarting from information sampled
    if e > 50: reward_history_conv.append(np.mean(rewards))
plt.plot(reward_history_conv)

In [ ]:
for iteration in trange(ITERATIONS):
    # get current state of the boards
    state = env_.to_state() # this is the state
    state=tf.reshape(state, (1000, ))
    original_probs=agent.actor(state)
    samples = tf.random.categorical(math.log(original_probs), 1)[:, 0]
    
    reward=[]
    with tf.GradientTape(persistent=True) as tape:
        """ 
        tensor of actions, consider that
            UP = 0
            RIGHT = 1
            DOWN = 2
            LEFT = 3
        """
        actions = samples
        rewards = env_.move(actions)
        new_state = tf.constant(env_.to_state())

        # calculate the loss of whichever algorithm you have picked
        loss = ...

    gradient = tape.gradient(..., ...)
    optimizer.apply_gradients(zip(gradient, ...))

 ### Random policy reward
 
Just a baseline (not the one you are supposed to develop)

In [ ]:
random_env = get_env(100)
random_rewards = []

for _ in trange(1000):
    probs = tf.convert_to_tensor([[.25]*4]*random_env.n_boards)
    #sample actions
    actions =  tf.random.categorical(tf.math.log(probs), 1, dtype=tf.int32)
    # MDP update
    rewards = random_env.move(actions)
    random_rewards.append(np.mean(rewards))

In [ ]:
random_rewards = []
env_ = get_env()

for _ in trange(1500):
    probs = tf.convert_to_tensor([[.25]*4]*env_.n_boards)
    #sample actions
    actions =  tf.random.categorical(tf.math.log(probs), 1, dtype=tf.int32)
    # MDP update
    rewards = env_.move(actions)
    random_rewards.append(np.mean(rewards))

In [ ]:
plt.plot(random_rewards)
plt.plot(reward_history_agent)
plt.plot(reward_history_conv)
plt.plot(reward_history_deep)